# Starting point
In this notebook I will for the first time try out PyTorch. I will stick to methods I have already tries out, but using PyTorch instead of Keras for deeplerning.


# The Data

In [1]:
import pandas as pd
import numpy as np
import torch

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

## Training set

In [2]:
data_df = pd.read_csv('train.csv')
digits = data_df.iloc[:, 1:].values
labels = data_df['label'].values

## Test set

In [3]:
test_df = pd.read_csv('test.csv')
digits_test = test_df.values

## Modify the data

Reshape the data.

In [265]:
img_dimensions = (1, 28, 28)

digits = digits.reshape(-1, *img_dimensions)
digits_test = digits_test.reshape(-1, *img_dimensions)

Scale the pixels.

In [340]:
from sklearn.preprocessing import Normalizer

In [469]:
digits_scaled = digits / 255
digits_test_scaled = digits_test / 255

## Validation set

In [470]:
from sklearn.model_selection import train_test_split
X, X_val, y, y_val = train_test_split(digits_scaled, labels, test_size = 8000, stratify = labels, random_state = 0)

## Convert to Tensors

In [471]:
# Training
X = torch.from_numpy(X).float()
X_val = torch.from_numpy(X_val).float()
y = torch.from_numpy(y).long()
y_val = torch.from_numpy(y_val).long()

# Test
X_test = torch.from_numpy(digits_test_scaled).float()

# Fully Connected Network
Let's start out basic with a fully connected network with two hidden layers.

In [472]:
import torch.nn.functional as F
import torch.nn as nn

In [503]:
class FullyConnected(nn.Module):
    
    def __init__(self):
        
        super(FullyConnected, self).__init__()
        
        self.fc1 = nn.Linear(28*28, 120)
        self.fc2 = nn.Linear(120, 120)
        self.output = nn.Linear(120, 10)
    
    def forward(self, x):
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.output(x)
        
        return x
        
        

In [504]:
fc_net = FullyConnected()

In [505]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fc_net.parameters())

In [506]:
X.reshape(-1, 28*28).shape

torch.Size([34000, 784])

In [507]:
import torch.utils.data

train_dataset = torch.utils.data.TensorDataset(X.reshape(-1, 28*28), y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)

validation_dataset = torch.utils.data.TensorDataset(X_val.reshape(-1, 28*28), y_val)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64)



In [508]:
from tqdm import tqdm

In [509]:
# Function inspired by https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def train_model(model, train_loader, optimizer, criterion, validation_loader = None, epochs = 2):
    
    # Only enter the validation state if there is a validation_loader
    phases = ['train']
    data_set_loaders = {'train' : train_loader, 'val' : validation_loader} 
    if validation_loader:
        phases.append('val')
        
    for epoch in range(epochs):
        
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-' * 10)

        for phase in phases:
            
            data_set_loader = data_set_loaders[phase]
            
            # Only update model weights based on the training data
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for i, batch in tqdm(enumerate(data_set_loader), 
                                 total = int(np.ceil(len(data_set_loader.dataset) / data_set_loader.batch_size))):
                inputs, labels = batch
                
                #labels = torch.autograd.Variable(labels).type(torch.LongTensor)

                optimizer.zero_grad()
                
                # Only track history during training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    predictions = torch.argmax(outputs, dim=1)
                    
                    # Only perform backpropagation during training
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # Save statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predictions == labels.data)
                
            epoch_loss = running_loss / len(data_set_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_set_loader.dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                
            

In [510]:
train_model(fc_net, train_loader, optimizer, criterion, validation_loader=validation_loader, epochs=10)

Epoch 1/10
----------


100%|███████████████████████████████████████████████████████████████████████████████| 532/532 [00:02<00:00, 231.22it/s]


train Loss: 0.3902 Acc: 0.8910


100%|███████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 460.79it/s]


val Loss: 0.2564 Acc: 0.9266
Epoch 2/10
----------


...

val Loss: 0.1490 Acc: 0.9606
Epoch 10/10
----------


100%|███████████████████████████████████████████████████████████████████████████████| 532/532 [00:02<00:00, 182.99it/s]


train Loss: 0.0214 Acc: 0.9944


100%|███████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 368.63it/s]


val Loss: 0.1421 Acc: 0.9650


Seems like it works okay! (Some serious overfitting goin on though...)

I'm not interested in doing hyper parameter tuning, so let's just check that it works okay on the test set as well.

In [511]:
fc_net.eval()

FullyConnected(
  (fc1): Linear(in_features=784, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=120, bias=True)
  (output): Linear(in_features=120, out_features=10, bias=True)
)

In [512]:
test_dataset = torch.utils.data.TensorDataset(X_test.reshape(-1, 28*28))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

In [513]:
len(test_dataset)

28000

In [514]:
predictions = []
for batch in test_loader:
    batch = batch[0]
    predictions += list(torch.argmax(fc_net(batch), dim=1))

In [515]:
submission_df = pd.DataFrame(list(zip(np.arange(1, 28001), map(lambda x: x.item(), predictions))), columns = ['ImageID', 'Label'])
submission_df.set_index('ImageID').to_csv('Submissions/pytorch_fc_1.csv')

Submission accuracy of 96.7% accuracy, which is similar to my validation accuracy. Great!

# CNN
Now let's build a Convolutional Network instead. I have done this once before using Keras, let's start by replicating the same model. (As inspired by the Keras teams example CNN for MNIST. [Github code](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py).)

In [486]:
class CNN(nn.Module):
    
    def __init__(self):
        
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 4, stride=2)
        self.conv2 = nn.Conv2d(32, 64, 2)
        
        self.dropout1 = nn.Dropout2d(p=.25)
        self.dropout2 = nn.Dropout(p=.5)
        
        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.output = nn.Linear(128, 10)
    
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.dropout1(x)
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.output(x)
        return x
        
        

In [487]:
cnn = CNN()

In [488]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters())

In [489]:
train_dataset = torch.utils.data.TensorDataset(X, y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128)

validation_dataset = torch.utils.data.TensorDataset(X_val, y_val)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=128)

In [490]:
train_model(cnn, train_loader, optimizer, criterion, validation_loader=validation_loader, epochs=2)

Epoch 1/2
----------


100%|████████████████████████████████████████████████████████████████████████████████| 266/266 [00:28<00:00,  9.28it/s]


train Loss: 0.5491 Acc: 0.8301


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 24.64it/s]


val Loss: 0.1496 Acc: 0.9547
Epoch 2/2
----------


100%|████████████████████████████████████████████████████████████████████████████████| 266/266 [00:25<00:00, 10.29it/s]


train Loss: 0.1760 Acc: 0.9481


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 24.47it/s]


val Loss: 0.0893 Acc: 0.9731


In [491]:
train_model(cnn, train_loader, optimizer, criterion, validation_loader=validation_loader, epochs=10)

Epoch 1/10
----------


100%|████████████████████████████████████████████████████████████████████████████████| 266/266 [00:28<00:00,  9.49it/s]


train Loss: 0.1242 Acc: 0.9629


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 23.35it/s]


val Loss: 0.0735 Acc: 0.9779
Epoch 2/10
----------


...

val Loss: 0.0490 Acc: 0.9855
Epoch 10/10
----------


100%|████████████████████████████████████████████████████████████████████████████████| 266/266 [00:40<00:00,  6.64it/s]


train Loss: 0.0487 Acc: 0.9841


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 21.74it/s]


val Loss: 0.0524 Acc: 0.9852


Cool, works similarily to the Keras variant!

I'm not that interested in tuning the hyperparameters of this model, so I'll just leave it here.

# Summary
In this notebook I have built two simple models for classification on MNIST using PyTorch. I have also built a simple training interface to be extended and reused in the future.

In the process I have learned a couple of things:
* How to build simple PyTorch models
* How to navigate the PyTorch documentation
* How to port a CNN architecture from Keras to PyTorch